# Assignment 9: GBDT

#### Response Coding: Example

<img src='http://i.imgur.com/TufZptV.jpg' width=700px>

> The response tabel is built only on train dataset.
> For a category which is not there in train data and present in test data, we will encode them with default values
Ex: in our test data if have State: D then we encode it as [0.5, 0.05]

<ol>
    <li><strong>Apply GBDT on these feature sets</strong>
        <ul>
            <li><font color='red'>Set 1</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF)+  preprocessed_eassay (TFIDF)+sentiment Score of eassay(check the bellow example, include all 4 values as 4 features)</li>
            <li><font color='red'>Set 2</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF W2V)+  preprocessed_eassay (TFIDF W2V)</li>        </ul>
    </li>
    <li><strong>The hyper paramter tuning (Consider any two hyper parameters)</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>find the best hyper paramter using k-fold cross validation/simple cross validation data</li>
    <li>use gridsearch cv or randomsearch cv or you can write your own for loops to do this task</li>
        </ul>
    </li>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/Gp2DQmh.jpg' width=500px> with X-axis as <strong>n_estimators</strong>, Y-axis as <strong>max_depth</strong>, and Z-axis as <strong>AUC Score</strong> , we have given the notebook which explains how to plot this 3d plot, you can find it in the same drive <i>3d_scatter_plot.ipynb</i></li>
            <p style="text-align:center;font-size:30px;color:red;"><strong>or</strong></p> <br>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/fgN9aUP.jpg' width=300px> <a href='https://seaborn.pydata.org/generated/seaborn.heatmap.html'>seaborn heat maps</a> with rows as <strong>n_estimators</strong>, columns as <strong>max_depth</strong>, and values inside the cell representing <strong>AUC Score</strong> </li>
    <li>You choose either of the plotting techniques out of 3d plot or heat map</li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='https://i.imgur.com/wMQDTFe.jpg' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='https://i.imgur.com/IdN5Ctv.png' width=300px></li>
            </ul>
    <br>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format
        <img src='http://i.imgur.com/YVpIGGE.jpg' width=400px>
    </li>
</ol>

In [0]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# import nltk
# nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

for_sentiment = 'a person is a person no matter how small dr seuss i teach the smallest students with the biggest enthusiasm \
for learning my students learn in many different ways using all of our senses and multiple intelligences i use a wide range\
of techniques to help all my students succeed students in my class come from a variety of different backgrounds which makes\
for wonderful sharing of experiences and cultures including native americans our school is a caring community of successful \
learners which can be seen through collaborative student project based learning in and out of the classroom kindergarteners \
in my class love to work with hands on materials and have many different opportunities to practice a skill before it is\
mastered having the social skills to work cooperatively with friends is a crucial aspect of the kindergarten curriculum\
montana is the perfect place to learn about agriculture and nutrition my students love to role play in our pretend kitchen\
in the early childhood classroom i have had several kids ask me can we try cooking with real food i will take their idea \
and create common core cooking lessons where we learn important math and writing concepts while cooking delicious healthy \
food for snack time my students will have a grounded appreciation for the work that went into making the food and knowledge \
of where the ingredients came from as well as how it is healthy for their bodies this project would expand our learning of \
nutrition and agricultural cooking recipes by having us peel our own apples to make homemade applesauce make our own bread \
and mix up healthy plants from our classroom garden in the spring we will also create our own cookbooks to be printed and \
shared with families students will gain math and literature skills as well as a life long enjoyment for healthy cooking \
nannan'
ss = sid.polarity_scores(for_sentiment)

for k in ss:
    print('{0}: {1}, '.format(k, ss[k]), end='')

# we can use these 4 things as features/attributes (neg, neu, pos, compound)
# neg: 0.0, neu: 0.753, pos: 0.247, compound: 0.93

D:\installed\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning:

The twython library has not been installed. Some functionality from the twitter package will not be available.



neg: 0.01, neu: 0.745, pos: 0.245, compound: 0.9975, 

<h1>1. GBDT (xgboost/lightgbm) </h1>

## 1.1 Loading Data

In [1]:
import pandas as pd
data = pd.read_csv('preprocessed_data.csv')

X = data.drop("project_is_approved",axis=1)
Y = data["project_is_approved"]

<h2>1.2 Splitting data into Train and cross validation(or test): Stratified Sampling</h2>

In [2]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label
    
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=15,test_size=0.33,stratify=Y)

print("Verfication after the splitting th data:",x_train.shape[0]+x_test.shape[0])

Verfication after the splitting th data: 109248


<h2>1.3 Make Data Model Ready: encoding essay</h2>

In [3]:
import pickle
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [15]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

## preparing set 1 data

from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import numpy as np

vectorizer = TfidfVectorizer(min_df=10,max_features=5000,ngram_range=(1,4))
x_train_essay_tfidf = vectorizer.fit_transform(x_train["essay"].values)
x_test_essay_tfidf = vectorizer.transform(x_test["essay"].values)

print("After TFIDF vectorization for feature: essay")
print(x_train_essay_tfidf.shape,y_train.shape)
print(x_test_essay_tfidf.shape,y_test.shape)
print("*"*50)

## preparing for set 2 data
## computing IDF Values and formming a dictionary of it for every unique value
tfidfvectorizer = TfidfVectorizer()
tfidfvectorizer.fit(x_train["essay"].values)
# forming a dictionary such that {feature_name:idf_value} ==> will be helpful later
# zip function will take only iterable objects
dictionary = dict(zip(tfidfvectorizer.get_feature_names(),list(tfidfvectorizer.idf_)))
tfidf_words = set(tfidfvectorizer.get_feature_names())

from tqdm import tqdm
x_train_essay_tfidf_w2v =[]
for sentence in tqdm(x_train["essay"].values):
    vector = np.zeros(300) # As glove data is 300 dim
    tfidf_weight = 0
    for word in sentence.split():
        if((word in glove_words) and (word in tfidf_words)):
            # will get the vector for every single word in a sentence from glove words pickl file
            vec = model[word] 
            # calculating tfidf value
            tfidf = dictionary[word]*(sentence.count(word)/len(sentence.split()))
            vector += (vec*tfidf)
            tfidf_weight += tfidf
    if(tfidf_weight!=0):
        vector/=tfidf_weight
    x_train_essay_tfidf_w2v.append(vector)
        
# Similarly for test data
x_test_essay_tfidf_w2v =[]
for sentence in tqdm(x_test["essay"].values):
    vector = np.zeros(300) # As glove data is 300 dim
    tfidf_weight = 0
    for word in sentence.split():
        vector
        if ((word in glove_words) and (word in tfidf_words)):
            vec = model[word]
            tfidf = dictionary[word]*(sentence.count(word)/len(sentence.split()))
            vector += (vec*tfidf)
            tfidf_weight += tfidf
    if(tfidf_weight!=0):
        vector/=tfidf_weight
    x_test_essay_tfidf_w2v.append(vector)

x_train_essay_tfidf_w2v = np.array(x_train_essay_tfidf_w2v)
x_test_essay_tfidf_w2v = np.array(x_test_essay_tfidf_w2v)
print("After TFIDF-W2V vectorization for feature: essay")
print(x_train_essay_tfidf_w2v.shape,y_train.shape)
print(x_test_essay_tfidf_w2v.shape,y_test.shape)
print("*"*50) 

After TFIDF vectorization for feature: essay
(73196, 5000) (73196,)
(36052, 5000) (36052,)
**************************************************


100%|██████████| 36052/36052 [00:50<00:00, 720.15it/s]


After TFIDF-W2V vectorization for feature: essay
(73196, 300) (73196,)
(36052, 300) (36052,)
**************************************************


<h2>1.4 Make Data Model Ready: encoding numerical, categorical features</h2>

In [16]:
import warnings
warnings.filterwarnings(action="ignore")

# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding 
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label
    
print("*"*20,"Categorical features","*"*20)
# performing response encoding for categorical features

# Teacher_prefix
x_train["project_is_approved"] = list(y_train) # adding y_train for checking probability values
den = x_train["teacher_prefix"].value_counts().to_dict() # for denominator of each row
x_train["teacher1"] = np.nan
x_train["teacher2"] = np.nan
x_test["teacher1"] = np.nan
x_test["teacher2"] = np.nan
for i in x_train["teacher_prefix"].unique():
    count1 = x_train["teacher_prefix"][(x_train["teacher_prefix"]==i)&(x_train["project_is_approved"]==1)].count()
    count2 = x_train["teacher_prefix"][(x_train["teacher_prefix"]==i)&(x_train["project_is_approved"]==0)].count()
    x_train["teacher1"][x_train["teacher_prefix"]==i]=count1/den[i]
    x_train["teacher2"][x_train["teacher_prefix"]==i]=count2/den[i]    

for i in x_test["teacher_prefix"].unique():
    if i in den.keys():
        count1 = x_train["teacher_prefix"][(x_train["teacher_prefix"]==i)&(x_train["project_is_approved"]==1)].count()
        count2 = x_train["teacher_prefix"][(x_train["teacher_prefix"]==i)&(x_train["project_is_approved"]==0)].count()
        x_test["teacher1"][x_test["teacher_prefix"]==i]=count1/den[i]
        x_test["teacher2"][x_test["teacher_prefix"]==i]=count2/den[i]
    else:
        x_test["teacher1"][x_test["teacher_prefix"]==i]=0.5
        x_test["teacher2"][x_test["teacher_prefix"]==i]=0.5
        
x_train_teacher_resp = np.asarray(x_train[["teacher1","teacher2"]]) 
x_test_teacher_resp = np.asarray(x_test[["teacher1","teacher2"]])

print("After Response coding of categorical feature: teacher_prefix")
print(x_train_teacher_resp.shape, y_train.shape)
print(x_test_teacher_resp.shape, y_test.shape)
print("="*50)

# project_grade_category
den = x_train["project_grade_category"].value_counts().to_dict() # for denominator of each row
x_train["proj1"] = np.nan
x_train["proj2"] = np.nan
x_test["proj1"] = np.nan
x_test["proj2"] = np.nan
for i in x_train["project_grade_category"].unique():
    count1 = x_train["project_grade_category"][(x_train["project_grade_category"]==i)&(x_train["project_is_approved"]==1)].count()
    count2 = x_train["project_grade_category"][(x_train["project_grade_category"]==i)&(x_train["project_is_approved"]==0)].count()
    x_train["proj1"][x_train["project_grade_category"]==i]=count1/den[i]
    x_train["proj2"][x_train["project_grade_category"]==i]=count2/den[i]    

for i in x_test["project_grade_category"].unique():
    if i in den.keys():
        count1 = x_train["project_grade_category"][(x_train["project_grade_category"]==i)&(x_train["project_is_approved"]==1)].count()
        count2 = x_train["project_grade_category"][(x_train["project_grade_category"]==i)&(x_train["project_is_approved"]==0)].count()
        x_test["proj1"][x_test["project_grade_category"]==i]=count1/den[i]
        x_test["proj2"][x_test["project_grade_category"]==i]=count2/den[i]
    else:
        x_test["proj1"][x_test["project_grade_category"]==i]=0.5
        x_test["proj2"][x_test["project_grade_category"]==i]=0.5

x_train_prj_grade_resp = np.asarray(x_train[["proj1","proj2"]]) 
x_test_prj_grade_resp = np.asarray(x_test[["proj1","proj2"]])

print("After Response coding of categorical feature: project_grade_category")
print(x_train_prj_grade_resp.shape, y_train.shape)
print(x_test_prj_grade_resp.shape, y_test.shape)
print("="*50)

# school_state
den = x_train["school_state"].value_counts().to_dict() # for denominator of each row
x_train["school1"] = np.nan
x_train["school2"] = np.nan
x_test["school1"] = np.nan
x_test["school2"] = np.nan
for i in x_train["school_state"].unique():
    count1 = x_train["school_state"][(x_train["school_state"]==i)&(x_train["project_is_approved"]==1)].count()
    count2 = x_train["school_state"][(x_train["school_state"]==i)&(x_train["project_is_approved"]==0)].count()
    x_train["school1"][x_train["school_state"]==i]=count1/den[i]
    x_train["school2"][x_train["school_state"]==i]=count2/den[i]    

for i in x_test["school_state"].unique():
    if i in den.keys():
        count1 = x_train["school_state"][(x_train["school_state"]==i)&(x_train["project_is_approved"]==1)].count()
        count2 = x_train["school_state"][(x_train["school_state"]==i)&(x_train["project_is_approved"]==0)].count()
        x_test["school1"][x_test["school_state"]==i]=count1/den[i]
        x_test["school2"][x_test["school_state"]==i]=count2/den[i]
    else:
        x_test["school1"][x_test["school_state"]==i]=0.5
        x_test["school2"][x_test["school_state"]==i]=0.5

x_train_school_resp = np.asarray(x_train[["school1","school2"]]) 
x_test_school_resp = np.asarray(x_test[["school1","school2"]])

print("After Response coding of categorical feature: school_state")
print(x_train_school_resp.shape, y_train.shape)
print(x_test_school_resp.shape, y_test.shape)
print("="*50)

#Clean categories
den = x_train["clean_categories"].value_counts().to_dict() # for denominator of each row
x_train["clean1"] = np.nan
x_train["clean2"] = np.nan
x_test["clean1"] = np.nan
x_test["clean2"] = np.nan
for i in x_train["clean_categories"].unique():
    count1 = x_train["clean_categories"][(x_train["clean_categories"]==i)&(x_train["project_is_approved"]==1)].count()
    count2 = x_train["clean_categories"][(x_train["clean_categories"]==i)&(x_train["project_is_approved"]==0)].count()
    x_train["clean1"][x_train["clean_categories"]==i]=count1/den[i]
    x_train["clean2"][x_train["clean_categories"]==i]=count2/den[i]    

for i in x_test["clean_categories"].unique():
    if i in den.keys():
        count1 = x_train["clean_categories"][(x_train["clean_categories"]==i)&(x_train["project_is_approved"]==1)].count()
        count2 = x_train["clean_categories"][(x_train["clean_categories"]==i)&(x_train["project_is_approved"]==0)].count()
        x_test["clean1"][x_test["clean_categories"]==i]=count1/den[i]
        x_test["clean2"][x_test["clean_categories"]==i]=count2/den[i]
    else:
        x_test["clean1"][x_test["clean_categories"]==i]=0.5
        x_test["clean2"][x_test["clean_categories"]==i]=0.5

x_train_cl_categories_resp = np.asarray(x_train[["clean1","clean2"]]) 
x_test_cl_categories_resp = np.asarray(x_test[["clean1","clean2"]])

print("After Response coding of categorical feature: clean_categories")
print(x_train_cl_categories_resp.shape, y_train.shape)
print(x_test_cl_categories_resp.shape, y_test.shape)
print("="*50)

#Clean sub-categories
den = x_train["clean_subcategories"].value_counts().to_dict() # for denominator of each row
x_train["cleansub1"] = np.nan
x_train["cleansub2"] = np.nan
x_test["cleansub1"] = np.nan
x_test["cleansub2"] = np.nan
for i in x_train["clean_subcategories"].unique():
    count1 = x_train["clean_subcategories"][(x_train["clean_subcategories"]==i)&(x_train["project_is_approved"]==1)].count()
    count2 = x_train["clean_subcategories"][(x_train["clean_subcategories"]==i)&(x_train["project_is_approved"]==0)].count()
    x_train["cleansub1"][x_train["clean_subcategories"]==i]=count1/den[i]
    x_train["cleansub2"][x_train["clean_subcategories"]==i]=count2/den[i]    

for i in x_test["clean_subcategories"].unique():
    if i in den.keys():
        count1 = x_train["clean_subcategories"][(x_train["clean_subcategories"]==i)&(x_train["project_is_approved"]==1)].count()
        count2 = x_train["clean_subcategories"][(x_train["clean_subcategories"]==i)&(x_train["project_is_approved"]==0)].count()
        x_test["cleansub1"][x_test["clean_subcategories"]==i]=count1/den[i]
        x_test["cleansub2"][x_test["clean_subcategories"]==i]=count2/den[i]
    else:
        x_test["cleansub1"][x_test["clean_subcategories"]==i]=0.5
        x_test["cleansub2"][x_test["clean_subcategories"]==i]=0.5

x_train_cl_subcategories_resp = np.asarray(x_train[["cleansub1","cleansub2"]]) 
x_test_cl_subcategories_resp = np.asarray(x_test[["cleansub1","cleansub2"]])

print("After Response coding of categorical feature: clean_subcategories")
print(x_train_cl_subcategories_resp.shape, y_train.shape)
print(x_test_cl_subcategories_resp.shape, y_test.shape)
print("="*50)

## Removing project_approved column from the x_train as all transformations are over ##
x_train.drop("project_is_approved",axis=1,inplace=True)

print("*"*20,"Numerical features","*"*20)
# Numerical features perform column standardzation
# Price
from sklearn.preprocessing import Normalizer
nm = Normalizer()
x_train_price_nrmlz = nm.fit_transform(x_train["price"].values.reshape(1,-1))
x_test_price_nrmlz = nm.transform(x_test["price"].values.reshape(1,-1))
x_train_price_nrmlz = x_train_price_nrmlz.reshape(-1,1)
x_test_price_nrmlz = x_test_price_nrmlz.reshape(-1,1)

print("After vectorizations of categorical feature: Price")
print(x_train_price_nrmlz.shape, y_train.shape)
print(x_test_price_nrmlz.shape, y_test.shape)
print("="*50)

# Number of projects previously done by teacher
nm = Normalizer()
x_train_num_proj_teacher_nrmlz = nm.fit_transform(x_train["teacher_number_of_previously_posted_projects"].values.reshape(1,-1))
x_test_num_proj_teacher_nrmlz = nm.transform(x_test["teacher_number_of_previously_posted_projects"].values.reshape(1,-1))
x_train_num_proj_teacher_nrmlz = x_train_num_proj_teacher_nrmlz.reshape(-1,1)
x_test_num_proj_teacher_nrmlz = x_test_num_proj_teacher_nrmlz.reshape(-1,1)

print("After vectorizations of categorical feature: teacher_number_of_previously_posted_projects")
print(x_train_num_proj_teacher_nrmlz.shape, y_train.shape)
print(x_test_num_proj_teacher_nrmlz.shape, y_test.shape)
print("="*50)           

******************** Categorical features ********************
After Response coding of categorical feature: teacher_prefix
(73196, 2) (73196,)
(36052, 2) (36052,)
After Response coding of categorical feature: project_grade_category
(73196, 2) (73196,)
(36052, 2) (36052,)
After Response coding of categorical feature: school_state
(73196, 2) (73196,)
(36052, 2) (36052,)
After Response coding of categorical feature: clean_categories
(73196, 2) (73196,)
(36052, 2) (36052,)
After Response coding of categorical feature: clean_subcategories
(73196, 2) (73196,)
(36052, 2) (36052,)
******************** Numerical features ********************
After vectorizations of categorical feature: Price
(73196, 1) (73196,)
(36052, 1) (36052,)
After vectorizations of categorical feature: teacher_number_of_previously_posted_projects
(73196, 1) (73196,)
(36052, 1) (36052,)


<h2>1.5 Make Data Model Ready: Sentimental analysis on essay feature and converting into four features</h2>

In [17]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

x_train_essay_senti = []
x_test_essay_senti = []

for ess in x_train["essay"].values:
    ss = sid.polarity_scores(ess)
    x_train_essay_senti.append(list(ss.values()))

for ess in x_test["essay"].values:
    ss = sid.polarity_scores(ess)
    x_test_essay_senti.append(list(ss.values()))
    
x_train_essay_senti = np.array(x_train_essay_senti)
x_test_essay_senti = np.array(x_test_essay_senti)

print(x_train_essay_senti.shape)
print(x_test_essay_senti.shape)

(73196, 4)
(36052, 4)


In [21]:
from scipy.sparse import hstack

x_tr_tfidf = hstack((x_train_essay_tfidf,x_train_teacher_resp,x_train_prj_grade_resp,x_train_school_resp,x_train_cl_categories_resp,x_train_cl_subcategories_resp,x_train_price_nrmlz,x_train_num_proj_teacher_nrmlz,x_train_essay_senti)).tocsr()
x_te_tfidf = hstack((x_test_essay_tfidf,x_test_teacher_resp,x_test_prj_grade_resp,x_test_school_resp,x_test_cl_categories_resp,x_test_cl_subcategories_resp,x_test_price_nrmlz,x_test_num_proj_teacher_nrmlz,x_test_essay_senti)).tocsr()

print("Final Data matrix for set1:")
print(x_tr_tfidf.shape, y_train.shape)
print(x_te_tfidf.shape, y_test.shape)
print("="*50)

x_tr_tfidf_w2v = hstack((x_train_essay_tfidf_w2v,x_train_teacher_resp,x_train_prj_grade_resp,x_train_school_resp,x_train_cl_categories_resp,x_train_cl_subcategories_resp,x_train_price_nrmlz,x_train_num_proj_teacher_nrmlz,x_train_essay_senti)).tocsr()
x_te_tfidf_w2v = hstack((x_test_essay_tfidf_w2v,x_test_teacher_resp,x_test_prj_grade_resp,x_test_school_resp,x_test_cl_categories_resp,x_test_cl_subcategories_resp,x_test_price_nrmlz,x_test_num_proj_teacher_nrmlz,x_test_essay_senti)).tocsr() 

print("Final Data matrix for set2:")
print(x_tr_tfidf_w2v.shape, y_train.shape)
print(x_te_tfidf_w2v.shape, y_test.shape)
print("="*50)

Final Data matrix for set1:
(73196, 5016) (73196,)
(36052, 5016) (36052,)


ValueError: could not broadcast input array from shape (73196,300) into shape (73196)

<h2>1.6 Appling Models on different kind of featurization as mentioned in the instructions</h2>

<br>Apply GBDT on different kind of featurization as mentioned in the instructions
<br> For Every model that you work on make sure you do the step 2 and step 3 of instrucations

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h1>3. Summary</h1>

<br> as mentioned in the step 4 of instructions